In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import time
import datetime as dt
import pickle
import numpy as np
from itertools import chain, combinations
import random
import scipy as sp
from __future__ import division
from dict_stops import * 
import pandas as pd
import os
import csv

In [3]:
frame = pd.read_csv('/home/cata/Documentos/Datois/etapas_2013_abril_allyearsids_10_100000.csv')

In [4]:
frame_2 = pd.read_csv('/home/cata/Documentos/Datois/etapas_2013_septiembre_allyearsids_10_100000.csv')

In [5]:
if os.name == 'nt':
    path_subway_dictionary = 'C:\Users\catalina\Documents\Datois\Diccionario-EstacionesMetro.csv'
    path_csv_sequences = 'C:\Users\catalina\Documents\sequences\\'
else:
    path_subway_dictionary = '/home/cata/Documentos/Datois/Diccionario-EstacionesMetro.csv'
    path_csv_sequences = '/home/cata/Documentos/sequences/'

# Función que carga las estaciones de metro
# en un diccionario
def load_metro_dictionary():
    dict_metro = {}
    with open(path_subway_dictionary,mode='r') as infile:
        reader = csv.reader(infile,delimiter=';')
        dict_metro = {rows[5]:rows[7] for rows in reader}
    return dict_metro


In [6]:
# Función que estandariza los valores de los paraderos de subida 
# y bajada
def update_vals(row,data = load_metro_dictionary()):
    if row.par_subida in data:
        row.par_subida = data[row.par_subida]
    if row.par_bajada in data:
        row.par_bajada = data[row.par_bajada]
    return row

In [7]:
# Función que estandariza los valores de los paraderos de subida 
# y bajada
def add_vals(row,latlong,paradero,data = dict_latlong_stops):
    stop_name = row[paradero]
    if stop_name in data:
        return data[stop_name][latlong]
    else :
        return np.nan

In [8]:
def frame_config(frame):
    frame['tiempo_subida'] = pd.to_datetime(frame.tiempo_subida)
    frame['tiempo_bajada'] = pd.to_datetime(frame.tiempo_bajada)
    frame = frame.apply(update_vals, axis=1)
    frame['weekday'] = frame.tiempo_subida.dt.dayofweek
    frame['lat_subida'] = frame.apply(add_vals,args=('lat','par_subida'),axis=1)
    frame['lat_bajada'] = frame.apply(add_vals,args=('lat','par_bajada'),axis=1)
    frame['long_subida'] = frame.apply(add_vals,args=('long','par_subida'),axis=1)
    frame['long_bajada'] = frame.apply(add_vals,args=('long','par_bajada'),axis=1)
    frame = frame.sort_values(by=['id', 'tiempo_subida'])
    frame['diferencia_tiempo'] = (frame['tiempo_subida']-frame['tiempo_subida'].shift()).fillna(0)
    return frame

In [9]:
def hour_to_seconds(an_hour):
    return int(an_hour.hour*3600 + an_hour.minute *60 + an_hour.second)

In [10]:
frame = frame_config(frame)

In [11]:
frame.head()

,tiempo_subida,id,x_subida,y_subida,tipo_transporte,serviciosentidovariante,tipo_dia,nviaje,netapa,x_bajada,...,par_bajada,zona_subida,zona_bajada,adulto,weekday,lat_subida,lat_bajada,long_subida,long_bajada,diferencia_tiempo
23,2013-04-14 06:45:44,1132106,348108.0,6289153.0,BUS,T203 00R,DOMINGO,1,1,346818.0,...,E-20-190-SN-40,328.0,307.0,0.0,6,-33.526277,-33.433786,-70.635551,-70.647786,0 days 00:00:00
22,2013-04-14 07:51:52,1132106,346751.0,6299389.0,BUS,T502 00I,DOMINGO,2,1,351363.0,...,T-15-135-PO-5,307.0,188.0,0.0,6,-33.434116,-33.406027,-70.648104,-70.598251,0 days 01:06:08
21,2013-04-14 19:56:47,1132106,351368.0,6302559.0,BUS,T502 00R,DOMINGO,3,1,346763.0,...,T-4-19-NS-100,188.0,55.0,0.0,6,-33.405971,-33.432332,-70.598379,-70.648651,0 days 12:04:55
20,2013-04-14 20:15:25,1132106,346713.0,6299427.0,BUS,T203 00I,DOMINGO,3,2,348095.0,...,T-24-205-NS-20,307.0,348.0,0.0,6,-33.433463,-33.525983,-70.649060,-70.635944,0 days 00:18:38
19,2013-04-15 21:04:59,1132106,348103.0,6289191.0,BUS,T206 00R,LABORAL,4,1,346844.0,...,T-20-190-SN-35,328.0,309.0,0.0,0,-33.526277,-33.434819,-70.635551,-70.647429,1 days 00:49:34


In [12]:
frame.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 23 to 99994
Data columns (total 23 columns):
tiempo_subida              100000 non-null datetime64[ns]
id                         100000 non-null int64
x_subida                   99224 non-null float64
y_subida                   99224 non-null float64
tipo_transporte            100000 non-null object
serviciosentidovariante    98074 non-null object
tipo_dia                   100000 non-null object
nviaje                     100000 non-null int64
netapa                     100000 non-null int64
x_bajada                   86774 non-null float64
y_bajada                   86774 non-null float64
tiempo_bajada              86774 non-null datetime64[ns]
par_subida                 99195 non-null object
par_bajada                 86767 non-null object
zona_subida                99180 non-null float64
zona_bajada                86764 non-null float64
adulto                     86774 non-null float64
weekday                    100

In [13]:
frame_2 = frame_config(frame_2)

## Paraderos sin latitud ni longitud

In [14]:
paraderos_sinlatlong = frame_2['par_subida'][frame_2['lat_subida'].isnull()& frame_2['par_subida'].notnull()].unique()

In [16]:
paraderos_sinlatlong

array(['T-20-304-SN-11', 'T-20-305-SN-10', 'T-20-207-OP-12',
       'T-20-400-OP-5', 'T-20-304-SN-13', 'L-33-71-OP-140', 'L-23-43-OP-1',
       'L-32-11-NS-60', 'L-29-35-SN-15', 'T-20-200-SN-54', 'L-23-22-SN-30',
       'L-33-84-PO-2', 'T-4-19-NS-33', 'L-4-3-NS-20', 'L-29-35-NS-20',
       'L-33-63-NS-5', 'T-4-19-SN-68', 'L-11-18-PO-20'], dtype=object)

In [84]:
frame_2 = frame_2[frame_2.lat_subida.notnull()]

In [54]:
from scipy.stats.mstats import mode
f = lambda x: mode(x, axis=None)[0][0]
g = lambda x: mode(x,axis=None)[1][0]
aggregations = {
    'tiempo_subida': "count"  # Calculate two results for the 'network' column with a list
    
}
a_group = frame.groupby(['par_subida','tipo_transporte']).agg(aggregations)
another_group = frame.groupby(['par_subida','par_bajada','lat_subida','tipo_transporte']).agg(aggregations)

In [57]:
sorted_group = another_group.sort_values('tiempo_subida',ascending=False)
sorted_group.head(100)

,,,,tiempo_subida
par_subida,par_bajada,lat_subida,tipo_transporte,
PLAZA DE ARMAS,PLAZA MAIPU,-33.437258,METRO,100
PLAZA MAIPU,PLAZA DE ARMAS,-33.510217,METRO,98
PLAZA DE PUENTE ALTO,TOBALABA,-33.609546,METRO,95
LA CISTERNA,ESCUELA MILITAR,-33.537712,METRO,84
LAS REJAS,TOBALABA,-33.457156,METRO,79
CAL Y CANTO,LA CISTERNA,-33.432885,METRO,69
MANQUEHUE,LA CISTERNA,-33.408844,METRO,61
HOSPITAL SOTERO DEL RIO,PLAZA DE ARMAS,-33.576460,METRO,59
LA CISTERNA,CAL Y CANTO,-33.537712,METRO,59


In [25]:
sorted_group.to_csv('od.csv')

In [28]:
another_group.head()

tiempo_subida
par_subida par_bajada               tipo_transporte               
ALCANTARA  BELLAS ARTES             METRO                        2
           BELLAVISTA DE LA FLORIDA METRO                        2
           CAL Y CANTO              METRO                        1
           CIUDAD DEL NINO          METRO                       11
           CUMMING                  METRO                        2

In [12]:
groupie_group = frame.groupby(['par_subida','tipo_transporte']).agg({'tiempo_subida':"count"})
sorted_par_subidas = groupie_group.sort_values('tiempo_subida',ascending=False)
sorted_par_subidas.to_csv('origin_.csv')
sorted_par_subidas.head()

,,tiempo_subida
par_subida,tipo_transporte,
LA CISTERNA,METRO,1382
TOBALABA,METRO,1361
MANQUEHUE,METRO,1116
UNIVERSIDAD DE CHILE,METRO,1057
PLAZA DE ARMAS,METRO,1022


In [13]:
groupie_group = frame.groupby(['par_bajada']).agg({'tiempo_subida':"count"})
sorted_par_bajadas = groupie_group.sort_values('tiempo_subida',ascending=False)
sorted_par_bajadas.to_csv('destination_.csv')
sorted_par_bajadas.head()

,tiempo_subida
par_bajada,
TOBALABA,1492
LA CISTERNA,1213
PLAZA DE ARMAS,1142
UNIVERSIDAD DE CHILE,953
ESCUELA MILITAR,929


In [46]:
sorted_par_subidas[sorted_par_subidas['tiempo_subida']>10].to_csv('origin_10.csv')
sorted_par_bajadas[sorted_par_bajadas['tiempo_subida']>10].to_csv('destination_10.csv')

In [15]:
with open('correct_and_wrong_indexs_alg1.pickle') as f:
    correct_alg1 =  pickle.load(f)
    wrong_alg1 = pickle.load(f)

In [16]:
with open('index_id_users.pickle') as f:
    users_id = pickle.load(f)
    

In [17]:
correct_alg1_ids = []
wrong_alg1_ids = []
for i in range(len(correct_alg1)):
    correct_alg1_ids.append(users_id[i])
for i in range(len(wrong_alg1)):
    wrong_alg1_ids.append(users_id[i])

In [30]:
def write_csv_grouped_data(a_frame,name,threshold):
    groupie_group = a_frame.groupby(['par_bajada','tipo_transporte']).agg({'tiempo_subida':"count"})
    sorted_par_bajadas = groupie_group.sort_values('tiempo_subida',ascending=False)
    sorted_par_bajadas[sorted_par_bajadas['tiempo_subida']>threshold].to_csv(name+'_'+str(threshold)+'_destination.csv')
    groupie_group = a_frame.groupby(['par_subida','tipo_transporte']).agg({'tiempo_subida':"count"})
    sorted_par_subidas = groupie_group.sort_values('tiempo_subida',ascending=False)
    sorted_par_subidas[sorted_par_subidas['tiempo_subida']>threshold].to_csv(name+'_'+str(threshold)+'_origin.csv')
    return sorted_par_subidas

### Guardar en csv viajes de los correctos e incorrectos

In [31]:
ff = write_csv_grouped_data(frame,'',0)

In [85]:
write_csv_grouped_data(frame_2,'',0)

,,tiempo_subida
par_subida,tipo_transporte,
TOBALABA,METRO,1448
LA CISTERNA,METRO,1408
MANQUEHUE,METRO,1186
PLAZA DE ARMAS,METRO,1144
UNIVERSIDAD DE CHILE,METRO,1143
PLAZA MAIPU,METRO,947
PEDRO DE VALDIVIA,METRO,938
SANTA ANA,METRO,935
ESCUELA MILITAR,METRO,905


In [37]:
frame

,tiempo_subida,id,x_subida,y_subida,tipo_transporte,serviciosentidovariante,tipo_dia,nviaje,netapa,x_bajada,...,par_bajada,zona_subida,zona_bajada,adulto,weekday,lat_subida,lat_bajada,long_subida,long_bajada,diferencia_tiempo
26,2013-09-23 20:58:46,1132106,348106.0,6289139.0,BUS,T203 00R,LABORAL,1,1,346824.0,...,E-20-190-SN-40,328.0,307.0,0.0,0,-33.526277,-33.433786,-70.635551,-70.647786,0 days 00:00:00
25,2013-09-23 21:24:25,1132106,346789.0,6299372.0,BUS,T502 00I,LABORAL,1,2,351366.0,...,T-15-135-PO-5,307.0,188.0,0.0,0,-33.434116,-33.406027,-70.648104,-70.598251,0 days 00:25:39
24,2013-09-24 07:15:40,1132106,351362.0,6302563.0,BUS,T502 00R,LABORAL,2,1,346661.0,...,T-4-19-NS-100,188.0,55.0,0.0,1,-33.405971,-33.432332,-70.598379,-70.648651,0 days 09:51:15
23,2013-09-24 08:00:08,1132106,346719.0,6299344.0,BUS,T206 06I,LABORAL,3,1,348078.0,...,T-24-205-NS-20,307.0,348.0,0.0,1,-33.433463,-33.525983,-70.649060,-70.635944,0 days 00:44:28
22,2013-09-24 09:13:22,1132106,347658.0,6289320.0,BUS,T352 00R,LABORAL,3,2,NaN,...,NaN,348.0,NaN,NaN,1,-33.524874,NaN,-70.640726,NaN,0 days 01:13:14
21,2013-09-24 21:12:10,1132106,348114.0,6289114.0,BUS,T203 00R,LABORAL,4,1,346811.0,...,E-20-190-SN-40,330.0,307.0,0.0,1,-33.529157,-33.433786,-70.635293,-70.647786,0 days 11:58:48
20,2013-09-24 21:41:09,1132106,346721.0,6299388.0,BUS,T502 00I,LABORAL,4,2,351366.0,...,T-15-135-PO-5,266.0,188.0,0.0,1,-33.433236,-33.406027,-70.653896,-70.598251,0 days 00:28:59
19,2013-09-25 06:53:03,1132106,351361.0,6302560.0,BUS,T502 00R,LABORAL,5,1,346734.0,...,T-4-19-NS-100,188.0,55.0,0.0,2,-33.405971,-33.432332,-70.598379,-70.648651,0 days 09:11:54
18,2013-09-25 07:08:48,1132106,346715.0,6299427.0,BUS,T203 00I,LABORAL,5,2,348095.0,...,T-24-205-NS-20,307.0,348.0,0.0,2,-33.433463,-33.525983,-70.649060,-70.635944,0 days 00:15:45
17,2013-09-25 20:51:51,1132106,348103.0,6289173.0,BUS,T203 00R,LABORAL,6,1,346821.0,...,E-20-190-SN-40,328.0,307.0,0.0,2,-33.526277,-33.433786,-70.635551,-70.647786,0 days 13:43:03


In [19]:
write_csv_grouped_data(frame[frame['id'].isin(wrong_alg1_ids)],'wrong_alg1',1)

In [20]:
write_csv_grouped_data(frame[frame['id'].isin(correct_alg1_ids)],'correct_alg1',1)

### Guardar en csv viajes de los correctos e incorrectos. sin transbordo

In [23]:
without_transbordors_frame = frame[frame['netapa']==1]

In [24]:
write_csv_grouped_data(without_transbordors_frame[without_transbordors_frame['id'].isin(wrong_alg1_ids)],'wrong_alg1_wo_tr',1)
write_csv_grouped_data(without_transbordors_frame[without_transbordors_frame['id'].isin(correct_alg1_ids)],'correct_alg1_wo_tr',1)